## Example of generating data and training the AA channel<br>


In [1]:
from multiprocessing import cpu_count
import os
os.environ["JULIA_NUM_THREADS"] = str(cpu_count())
from juliacall import Main as jl
jl.Pkg.activate("/home/zaldivar/Documents/Androniki/Github/GWBackFinder.jl")
jl.seval("using GWBackFinder")
from tqdm import tqdm
import sys
import numpy as np
sys.path.append("/home/zaldivar/Documents/Androniki/Github/GWBackFinder_python")
from src.GWBackFinder import train as GW_train
from src.GWBackFinder import prior as GW_prior
import torch

  Activating project at `~/Documents/Androniki/Github/GWBackFinder.jl`
[ Info: Precompiling GWBackFinder [7fa44c00-b40e-49bf-9bf3-4f6997ab6254]
/home/zaldivar/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



#### load the prior and sample 1 million points

In [2]:
custom_prior=GW_prior.get_prior()
z=custom_prior.sample(1000000)
#z=np.save("./samples.npy",z)
# %%

/home/zaldivar/.local/lib/python3.9/site-packages/sbi/utils/user_input_checks_utils.py:69: UserWarning: Prior is lacking mean attribute, estimating prior mean from samples.
  warnings.warn(
/home/zaldivar/.local/lib/python3.9/site-packages/sbi/utils/user_input_checks_utils.py:80: UserWarning: Prior is lacking variance attribute, estimating prior variance from
                samples...
  warnings.warn(


### Define frequency range and split in 26 bins of equal log-spacing. $idx_{26}$ shows in which bin the frequency belongs to and $logbins_{26}$ which are the boundaries of the bins. After we coarse grain the frequencies from $3 \cdot 10^{-3}$ Hz to the maximum frequency 0.5 Hz, i.e., we bin them in 1000 intervals of equal log-spacing. $idx$ shows in which of the 1000 bins the frequency belongs to and $logbins$ the boundaries of the 1000 bins.

In [3]:
f=jl.range(3*1e-5, 0.5, step=1e-6)
idx,idx26,logbins_26,logbins,f_filtered = jl.GWBackFinder.binning(f)
Sb1, Sb2, Sb3, Sb4, Sb5, Sb6, Sb7, Sb8, Sb9, Sb10, Sb11, Sb12, Sb13, Sb14, Sb15, Sb16, Sb17, Sb18, Sb19, Sb20, Sb21, Sb22, Sb23, Sb24, Sb25, Sb26 = logbins_26[0],logbins_26[1], logbins_26[2], logbins_26[3], logbins_26[4], logbins_26[5], logbins_26[6], logbins_26[7], logbins_26[8], logbins_26[9], logbins_26[10], logbins_26[11], logbins_26[12], logbins_26[13], logbins_26[14], logbins_26[15], logbins_26[16], logbins_26[17], logbins_26[18], logbins_26[19], logbins_26[20], logbins_26[21], logbins_26[22], logbins_26[23], logbins_26[24], logbins_26[25]

In [8]:
np.array(z[0,:])

array([0.6697098 , 0.99768716, 0.67019755, 0.5097361 , 0.2750281 ,
       0.22789496, 0.23268265, 0.9259667 , 0.10996038, 0.9437534 ,
       0.78090566, 0.26876634, 0.00523132, 0.5179544 , 0.3963638 ,
       0.32717127, 0.5350558 , 0.22476536, 0.55392593, 0.9559137 ,
       0.05114996, 0.73442805, 0.36213464, 0.5544201 , 0.17582512,
       0.8085161 , 0.7835821 , 3.3225775 ], dtype=float32)

### generate training data using the julia package

In [9]:
gw_total_list=[]
for i in tqdm(range((1))):
    Data_total = jl.GWBackFinder.model_train_data(np.array(z[i,:]), f, idx,
                                                  f_filtered,logbins, Sb1, Sb2, Sb3, Sb4, Sb5, Sb6, Sb7, Sb8, Sb9, Sb10, 
                                                  Sb11, Sb12, Sb13, Sb14, Sb15, Sb16, Sb17, Sb18, Sb19, Sb20, Sb21, Sb22, Sb23, Sb24, Sb25)

  0%|          | 0/1 [00:02<?, ?it/s]


JuliaError: Out of GPU memory
Stacktrace:
  [1] throw_api_error(res::CUDA.cudaError_enum)
    @ CUDA ~/.julia/packages/CUDA/YIj5X/lib/cudadrv/libcuda.jl:25
  [2] check
    @ ~/.julia/packages/CUDA/YIj5X/lib/cudadrv/libcuda.jl:34 [inlined]
  [3] cuDevicePrimaryCtxRetain
    @ ~/.julia/packages/CUDA/YIj5X/lib/utils/call.jl:26 [inlined]
  [4] CuContext
    @ ~/.julia/packages/CUDA/YIj5X/lib/cudadrv/context.jl:57 [inlined]
  [5] context(dev::CUDA.CuDevice)
    @ CUDA ~/.julia/packages/CUDA/YIj5X/lib/cudadrv/state.jl:253
  [6] CUDA.TaskLocalState(dev::CUDA.CuDevice)
    @ CUDA ~/.julia/packages/CUDA/YIj5X/lib/cudadrv/state.jl:50
  [7] CUDA.TaskLocalState()
    @ CUDA ~/.julia/packages/CUDA/YIj5X/lib/cudadrv/state.jl:50
  [8] task_local_state!()
    @ CUDA ~/.julia/packages/CUDA/YIj5X/lib/cudadrv/state.jl:79
  [9] #alloc#1017
    @ ~/.julia/packages/CUDA/YIj5X/src/pool.jl:0 [inlined]
 [10] alloc
    @ ~/.julia/packages/CUDA/YIj5X/src/pool.jl:411 [inlined]
 [11] CUDA.CuArray{Float32, 1, CUDA.

In [12]:
 Data_total 

0.003000000000000001

In [7]:
logbins[0]

0.003000000000000001

In [4]:
gw_total_list=[]
for i in tqdm(range((1000))):
    z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21, z22, z23, z24, z25, z26 = -12 +z[i,0:26].numpy()*(12-(-12))                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
    Amp =-14+z[i,26].numpy()*(-6-(-14))
    A = z[i,27].numpy()
    Data_total = jl.GWBackFinder.model_train_data(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, 
                                                  z16, z17, z18, z19, z20, z21, z22, z23, z24, z25, z26, Amp, A, f, idx,
                                                  f_filtered,logbins, Sb1, Sb2, Sb3, Sb4, Sb5, Sb6, Sb7, Sb8, Sb9, Sb10, 
                                                  Sb11, Sb12, Sb13, Sb14, Sb15, Sb16, Sb17, Sb18, Sb19, Sb20, Sb21, Sb22, Sb23, Sb24, Sb25)
    gw_total_list.append(np.array(Data_total))

  0%|          | 0/1000 [00:00<?, ?it/s]

### convert to tensor 

In [ ]:
gw_total=torch.tensor(np.array(gw_total_list) , dtype=torch.float32)
thetas=torch.tensor(z[0:len(z)])
# %%

### check shapes

In [ ]:
print(gw_total.shape)
print(thetas.shape)


### train and save inference in the file train_200.pkl

In [ ]:
GW_train.train(thetas=thetas, gw_total=gw_total, prior=custom_prior, resume_training=False, validation_fraction=0.2, 
          learning_rate=1e-4, show_train_summary=True, max_num_epochs=200, 
          path_saved=None, path_inference="/data/users/Androniki/", name_file="train_200.pkl", model_type="nsf", hidden_features=64, num_transforms=3)


### get posterior from the train_200.pkl and save it in posterior.pkl

In [ ]:
GW_train.get_posterior("/data/users/Androniki/train_200.pkl","posterior.pkl")